In [1]:
import math
import random
import time
import numpy
import pickle

from itertools import permutations

In [2]:
class Circle:

    def __init__(self, x=0.0, y=0.0, r=1.0):

        self.x = x
        self.y = y
        self.r = r

In [3]:
def get_search_space_domain(input_circle_list):

    all_domain = permutations(input_circle_list, len(input_circle_list))
    all_unique_domain = list(set(all_domain))
    seen = set()
    all_unique_domain_removed_reversed = [x for x in all_unique_domain if tuple(x[::-1]) not in seen and not seen.add(tuple(x))]

    search_space = all_unique_domain_removed_reversed.copy()

    return search_space

In [4]:
def arrange_circles_and_get_min_width_box(input_circles):

    l_circle_obj = []

    for i, circle_r in enumerate(input_circles):
        l_circle_obj.append(Circle(r=circle_r))

    base_circle = l_circle_obj[0]

    mr_points = []
    ml_points = []
    xs = []
    ys = []
    rs = []

    mr_points.append(2 * base_circle.r)
    ml_points.append(0)
    xs.append(base_circle.r)
    ys.append(base_circle.r)
    rs.append(base_circle.r)
    min_width = 2 * base_circle.r     # max(input_circles) * len(input_circles) * 2

    for circle in l_circle_obj[1:]:
        y2 = circle.r
        r2 = circle.r

        l_x2 = []
        for x1, y1, r1 in zip(xs, ys, rs):
            x2 = math.sqrt((r1 + r2) ** 2 - (y2 - y1) ** 2) + x1
            l_x2.append(x2)
            mr_points.append(x2 + r2)
            ml_points.append(x2 - r2)

        xs.append(max(l_x2))
        ys.append(y2)
        rs.append(r2)

        temp_min_width = abs(min(ml_points)) + max(mr_points)

        if temp_min_width > min_width:
            min_width = temp_min_width

    return min_width

In [5]:
def get_brute_force_solution(search_space):

    l_width_box = []

    for circles in search_space:
        width_box = arrange_circles_and_get_min_width_box(circles)
        l_width_box.append(width_box)

    min_width_box = min(l_width_box)
    min_width_index = l_width_box.index(min_width_box)
    arrangement_circles = search_space[min_width_index]

    return min_width_box, arrangement_circles

In [10]:
def get_greedy_solution(input_circle_list):

    sorted_input_circles = sorted(input_circle_list)
    min_width_box = arrange_circles_and_get_min_width_box(sorted_input_circles)

    min_width_box=min_width_box; arrangement_circles=sorted_input_circles

    return min_width_box, arrangement_circles

In [ ]:
def get_tabu_search_solution(input_circle_list):

    sorted_input_circles = sorted(input_circle_list)
    min_width_box = arrange_circles_and_get_min_width_box(sorted_input_circles)

    min_width_box=min_width_box; arrangement_circles=sorted_input_circles

    return min_width_box, arrangement_circles

In [ ]:
def generate_neighborhood_structure_for_ils_1_opt(current_solution):

    # ref: http://www.cleveralgorithms.com/nature-inspired/stochastic/iterated_local_search.html
    def swap_position(list_, pos1, pos2):

        list_[pos1], list_[pos2] = list_[pos2], list_[pos1]
        return list_

    l_curr_solution = list(current_solution)
    l_curr_solution_ori = l_curr_solution.copy()
    l_neighbors = []
    len_curr_sol = len(l_curr_solution_ori)

    for i in range(0, len_curr_sol-1):
        for j in range(i+1, len_curr_sol):
            current_solution = l_curr_solution_ori.copy()
            neighbor = swap_position(current_solution, i, j)
            l_neighbors.append(neighbor)

    if l_neighbors.__contains__(l_curr_solution_ori):
        l_neighbors.remove(l_curr_solution_ori)

    l_neighbors = [list(x) for x in set(tuple(x) for x in l_neighbors)]

    return l_neighbors

def get_iterated_local_search_solution(search_space, iteration_num=100):

    best_solution_width = 2 * max(list(search_space[0])) * len(list(search_space[0]))
    best_solution = list(search_space[0])

    for iter_num in range(iteration_num):

        init_solution = list(random.choice(search_space))
        init_width_box = arrange_circles_and_get_min_width_box(init_solution)
        current_solution = init_solution

        local_best_solution = init_solution
        local_best_solution_width = init_width_box

        while True:

            neighbors = generate_neighborhood_structure_for_ils_1_opt(current_solution)

            l_neighbor_width = []
            for neighbor in neighbors:
                min_width_box = arrange_circles_and_get_min_width_box(neighbor)
                l_neighbor_width.append(min_width_box)

            local_best_neighbor_width = min(l_neighbor_width)
            local_best_neighbor = neighbors[l_neighbor_width.index(local_best_neighbor_width)]

            if local_best_neighbor_width < local_best_solution_width:
                local_best_solution_width = local_best_neighbor_width
                current_solution = local_best_neighbor.copy()
                local_best_solution = local_best_neighbor

            else:
                break

        if local_best_solution_width < best_solution_width:
            best_solution = local_best_solution
            best_solution_width = local_best_solution_width

    min_width_box = best_solution_width
    arrangement_circles = best_solution

    return min_width_box, arrangement_circles

In [ ]:
def local_search_for_vns_with_2_opt(current_solution):

    def swap_position(list_, pos1, pos2):

        list_[pos1], list_[pos2] = list_[pos2], list_[pos1]
        return list_

    l_curr_solution = list(current_solution)
    l_curr_solution_ori = l_curr_solution.copy()
    l_neighbors = []
    len_curr_sol = len(l_curr_solution_ori)

    for i in range(len_curr_sol-1):

        for j in range(i+1, len_curr_sol):

            current_solution = l_curr_solution_ori.copy()
            neighbor_1_opt = swap_position(current_solution, i, j)

            indices = list(range(0, len(current_solution)))
            indices.remove(i)
            indices.remove(j)
            neighbor_1_opt_ori = neighbor_1_opt.copy()

            for k in indices[0:-1]:
                index_k = indices.index(k)
                for l in indices[index_k+1:]:
                    neighbor_1_opt = neighbor_1_opt_ori.copy()
                    neighbor = swap_position(neighbor_1_opt, k, l)
                    l_neighbors.append(neighbor)

    if l_neighbors.__contains__(l_curr_solution_ori):
        l_neighbors.remove(l_curr_solution_ori)

    l_neighbors = [list(x) for x in set(tuple(x) for x in l_neighbors)]

    return l_neighbors

def generate_neighborhood_for_vns_with_2_opt(current_solution, k):

    def swap_position(list_, pos1, pos2):

        list_[pos1], list_[pos2] = list_[pos2], list_[pos1]
        return list_

    for i in range(k):
        four_indices = numpy.random.choice(range(len(current_solution)), size=4, replace=False)
        current_solution = swap_position(current_solution, four_indices[0], four_indices[1])
        current_solution = swap_position(current_solution, four_indices[2], four_indices[3])

    return current_solution

def get_variable_neighborhood_search_solution(search_space, iteration_num=100, neighborhoods=50):

    best_solution_width = 2 * max(list(search_space[0])) * len(list(search_space[0]))
    best_solution = list(search_space[0])

    for iter_num in range(iteration_num):

        init_solution = list(random.choice(search_space))
        init_width_box = arrange_circles_and_get_min_width_box(init_solution)
        current_solution = init_solution

        local_best_solution = init_solution
        local_best_solution_width = init_width_box

        k = 1
        while k < neighborhoods:

            neighbor = generate_neighborhood_for_vns_with_2_opt(current_solution, k)
            neighbors = local_search_for_vns_with_2_opt(neighbor)

            l_neighbor_width = []
            for neighbor in neighbors:
                min_width_box = arrange_circles_and_get_min_width_box(neighbor)
                l_neighbor_width.append(min_width_box)

            local_best_neighbor_width = min(l_neighbor_width)
            local_best_neighbor = neighbors[l_neighbor_width.index(local_best_neighbor_width)]

            if local_best_neighbor_width < local_best_solution_width:
                local_best_solution_width = local_best_neighbor_width
                current_solution = local_best_neighbor.copy()
                local_best_solution = local_best_neighbor

            else:
                k+=1

        if local_best_solution_width < best_solution_width:
            best_solution = local_best_solution
            best_solution_width = local_best_solution_width

    min_width_box=best_solution_width
    arrangement_circles=best_solution

    return min_width_box, arrangement_circles

In [ ]:
def get_neighborhood_for_sa_with_2_opt(current_solution):

    def swap_position(list_, pos1, pos2):

        list_[pos1], list_[pos2] = list_[pos2], list_[pos1]
        return list_

    four_indices = numpy.random.choice(range(len(current_solution)), size=4, replace=False)
    current_solution = swap_position(current_solution, four_indices[0], four_indices[1])
    current_solution = swap_position(current_solution, four_indices[2], four_indices[3])

    return current_solution

def get_simulated_annealing_solution(search_space, T=1000000, r=0.9, MAX_TRIES=100):

    init_solution = list(search_space[0])
    best_solution = init_solution.copy()
    best_solution_width = arrange_circles_and_get_min_width_box(best_solution)

    tries = 0
    while tries < MAX_TRIES:

        init_solution = list(random.choice(search_space))
        current_solution = init_solution
        current_solution_width = arrange_circles_and_get_min_width_box(current_solution)

        t = T

        while t > 0.1:

            neighbor_solution = get_neighborhood_for_sa_with_2_opt(current_solution)
            neighbor_width_box = arrange_circles_and_get_min_width_box(list(neighbor_solution))

            if neighbor_width_box < current_solution_width:
                current_solution_width = neighbor_width_box

            else:
                q = random.uniform(0, 1)
                rand_change_val = (math.e ** ((current_solution_width - neighbor_width_box)/T))

                if q < rand_change_val:
                    current_solution = neighbor_solution

            t = t * r

        if current_solution_width < best_solution_width:
            best_solution_width = current_solution_width
            best_solution = current_solution

        tries += 1

    min_width_box=best_solution_width
    arrangement_circles=best_solution

    return min_width_box, arrangement_circles

In [6]:
l_results = []
limit_size = [5, 5, 7, 7, 10, 10, 13, 13]
range_complexities = [(1, 10), (20, 30), (1, 10), (20, 30), (1, 10), (20, 30), (1, 10), (20, 30)]

In [7]:
l_results = []
limit_size = [5]
range_complexities = [(1, 10)]

In [9]:
for size, comlexity in zip(limit_size, range_complexities):

    d_result = {}
    
    ###############################
    start = comlexity[0]; stop = comlexity[1]
    input_circle_list = [random.randint(start, stop) for iter in range(size)]
    print("\ninput_circles: ", input_circle_list)
    d_result["size"] = size
    d_result["comlexity"] = comlexity
    ###############################
    search_space = get_search_space_domain(input_circle_list)
    d_result["total_search_space_size"] = len(search_space)
    print("search_space_len: ", len(search_space))
    ############################################################################################################################
    start_time = time.time()
    brute_force_min_width_box, brute_force_arrangement_circles = get_brute_force_solution(search_space)
    brute_force_performance = time.time() - start_time
    print("brute_force_performance: ", brute_force_performance)
    
    d_result["bf_min_width_box"] = brute_force_min_width_box
    d_result["bf_arrangement_circles"] = brute_force_arrangement_circles
    d_result["bf_performance"] = brute_force_performance
    l_results.append(d_result)
    ############################################################################################################################


input_circles:  [9, 10, 10, 2, 1]
search_space_len:  30
brute_force_performance:  0.0


In [ ]:
l_results = []
limit_size = [7, 7, 10, 10]
range_complexities = [(1, 10), (20, 30), (1, 10), (20, 30)]

for size, comlexity in zip(limit_size, range_complexities):

    d_result = {}

    start = comlexity[0]; stop = comlexity[1]
    input_circle_list = [random.randint(start, stop) for iter in range(size)]
    print("\ninput_circles: ", input_circle_list)
    d_result["size"] = size
    d_result["comlexity"] = comlexity

    # start_time = time.time()
    # ts_min_width_box, ts_arrangement_circles = get_tabu_search_solution(input_circle_list)
    # ts_performance = time.time() - start_time
    #
    # d_result["ts_min_width_box"] = ts_min_width_box
    # d_result["ts_arrangement_circles"] = ts_arrangement_circles
    # d_result["ts_performance"] = ts_performance
    # l_results.append(d_result)

    search_space = get_search_space_domain(input_circle_list)
    d_result["total_search_space_size"] = len(search_space)
    print("search_space_len: ", len(search_space))

    start_time = time.time()
    vns_min_width_box, vns_arrangement_circles = get_variable_neighborhood_search_solution(search_space, iteration_num=100, neighborhoods=50)
    vns_performance = time.time() - start_time
    print("vns_performance: ", vns_performance)

    d_result["vns_min_width_box"] = vns_min_width_box
    d_result["vns_arrangement_circles"] = vns_arrangement_circles
    d_result["vns_performance"] = vns_performance
    l_results.append(d_result)

    start_time = time.time()
    sa_min_width_box, sa_arrangement_circles = get_simulated_annealing_solution(search_space, T=1000000, r=0.9, MAX_TRIES=100)
    sa_performance = time.time() - start_time
    print("sa_performance: ", sa_performance)

    d_result["sa_min_width_box"] = sa_min_width_box
    d_result["sa_arrangement_circles"] = sa_arrangement_circles
    d_result["sa_performance"] = sa_performance
    l_results.append(d_result)

    start_time = time.time()
    ils_min_width_box, ils_arrangement_circles = get_iterated_local_search_solution(search_space, iteration_num=100)
    ils_performance = time.time() - start_time
    print("ils_performance: ", ils_performance)

    d_result["ils_min_width_box"] = ils_min_width_box
    d_result["ils_arrangement_circles"] = ils_arrangement_circles
    d_result["ils_performance"] = ils_performance
    l_results.append(d_result)

    start_time = time.time()
    greedy_min_width_box, greedy_arrangement_circles = get_greedy_solution(input_circle_list)
    greedy_performance = time.time() - start_time
    print("greedy_performance: ", greedy_performance)

    d_result["gr_min_width_box"] = greedy_min_width_box
    d_result["gr_arrangement_circles"] = greedy_arrangement_circles
    d_result["gr_performance"] = greedy_performance
    l_results.append(d_result)

    start_time = time.time()
    brute_force_min_width_box, brute_force_arrangement_circles = get_brute_force_solution(search_space)
    brute_force_performance = time.time() - start_time
    print("brute_force_performance: ", brute_force_performance)

    d_result["bf_min_width_box"] = brute_force_min_width_box
    d_result["bf_arrangement_circles"] = brute_force_arrangement_circles
    d_result["bf_performance"] = brute_force_performance
    l_results.append(d_result)

    with open("l_results_.pkl", 'wb') as filehandle:
        pickle.dump(l_results, filehandle)